Chern insulator is characterized by Chern number, which is defined as
\begin{equation}
\begin{aligned}
C & =\sum_n^{o c c .} \frac{1}{2 \pi} \int_{\mathrm{BZ}} d k_x d k_y F_n=0, \pm 1, \pm 2, \pm 3, \cdots \\
F_n & =\left(\nabla \times \mathbf{A}_n\right)_z, \mathbf{A}_n=i\left\langle u_{n \mathbf{k}}\right| \frac{\partial}{\partial \mathbf{k}}\left|u_{n \mathbf{k}}\right\rangle
\end{aligned}
\end{equation}

$C=0$ corresponds to a trivial state, while $C=\pm 1, \pm2, ...$ correspond to nontrivial state. 

The Fukui-Hatsugai method is a numerical approach to compute the Chern number on a discrete momentum space (Brillouin zone) grid. It is particularly useful in tight-binding models. The method works by defining a lattice discretization of the Berry curvature and summing over a discrete Brillouin zone.

In [2]:
import numpy as np
import cmath

def QWZ_hamiltonian(kx, ky, t=1, m=-1):
    """
    QWZ model Hamiltonian (2 x 2).
    kx, ky: momentum components
    t: hopping amplitude
    m: mass term
    """
    sigma_x = np.array([[0, 1], 
                        [1, 0]], dtype=complex)
    sigma_y = np.array([[0, -1j], 
                        [1j,  0]], dtype=complex)
    sigma_z = np.array([[1,  0], 
                        [0, -1]], dtype=complex)
    
    # 对应系数
    Hx = t * np.sin(kx)           # 系数乘 sigma_x
    Hy = t * np.sin(ky)           # 系数乘 sigma_y
    Hz = m + t * np.cos(kx) + t * np.cos(ky)  # 系数乘 sigma_z

    # 构造 2x2 哈密顿量
    H = Hx * sigma_x + Hy * sigma_y + Hz * sigma_z
    return H


def fukui_hatsugai_chern(hamiltonian, Nk=20, band_index=0):
    """
    Compute the Chern number for a given band using the Fukui-Hatsugai method.
    Parameters:
        hamiltonian(kx, ky): returns an (n x n) Hermitian matrix
        Nk: number of k-points in each direction
        band_index: which band (0-based) to compute the Chern number for
    Returns:
        Chern number (float)
    """
    # Discretize the Brillouin zone
    dtheta = 2.0 * np.pi / Nk
    total_flux = 0.0

    for i in range(Nk):
        for j in range(Nk):
            # Define the four corners of the plaquette
            kx = i * dtheta
            ky = j * dtheta

            # Overlaps U12, U23, U34, U41
            # We'll define a helper function to get the normalized eigenvector
            # of the chosen band at (kx, ky).
            def eigvec(kx_, ky_):
                H = hamiltonian(kx_, ky_)
                vals, vecs = np.linalg.eig(H)
                # Sort eigenvalues, pick the eigenvector for band_index
                idx = np.argsort(vals)
                return vecs[:, idx[band_index]]

            u1 = eigvec(kx, ky)
            u2 = eigvec(kx + dtheta, ky)
            u3 = eigvec(kx + dtheta, ky + dtheta)
            u4 = eigvec(kx, ky + dtheta)

            # Compute link variables as determinants or inner products
            # For a single band, the link variable is just the inner product
            U12 = np.vdot(u1, u2)  # k -> k + d(kx)
            U23 = np.vdot(u2, u3)  # k + d(kx) -> k + d(kx) + d(ky)
            U34 = np.vdot(u3, u4)  # k + d(ky) -> k + d(kx) + d(ky)
            U41 = np.vdot(u4, u1)  # k + d(ky) -> k

            # Berry curvature on this plaquette
            # F = Im[ log(U12 * U23 * U34 * U41 ) ]
            # Note: Use cmath.log to handle complex logs
            prod = U12 * U23 * U34 * U41
            flux = np.angle(prod)  # same as Im(log(prod))

            total_flux += flux

    # Sum of flux over all plaquettes, divided by 2*pi
    chern_number = total_flux / (2.0 * np.pi)
    return chern_number

if __name__ == "__main__":
    # Example usage: QWZ model
    Nk = 100  # number of points in each direction
    C = fukui_hatsugai_chern(QWZ_hamiltonian, Nk=Nk, band_index=0)
    print(f"Computed Chern number = {C:.4f}")


Computed Chern number = -1.0000
